In [ ]:
import pandas as pd
import joblib
import os

In [ ]:
base = pd.read_csv('../data/raw/desafio_indicium_imdb.csv')

In [ ]:
base = base[base['Released_Year'] != 'PG']
base['Released_Year'] = base['Released_Year'].astype(int)

In [ ]:
base['Gross'] = base['Gross'].str.replace(r'[^0-9]','',regex=True)
base['Gross'] = base['Gross'].astype(float)

In [ ]:
base['Gross'] = base['Gross'].fillna(base['Gross'].median())
base['Meta_score'] = base['Meta_score'].fillna(base['Meta_score'].median())

In [ ]:
base['Runtime'] = base['Runtime'].str.replace(r'[^0-9]','', regex=True) 
base['Runtime'] = base['Runtime'].astype(int)
base.rename(columns={'Runtime':'Runtime_Min'},inplace=True) 

In [ ]:
novo_filme = [{
'Series_Title': 'The Shawshank Redemption', 
'Released_Year': '1994', 
'Certificate': 'A', 
'Runtime': '142 min', 
'Genre': 'Drama', 
'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 
'Meta_score': 80.0, 
'Director': 'Frank Darabont', 
'Star1': 'Tim Robbins', 
'Star2': 'Morgan Freeman', 
'Star3': 'Bob Gunton', 
'Star4': 'William Sadler', 
'No_of_Votes': 2343110, 
'Gross': '28,341,469'}] 

df = pd.DataFrame(novo_filme)
df['Runtime'] = df['Runtime'].str.replace(r'[^0-9]','', regex=True) 
df['Runtime'] = df['Runtime'].astype(int)
df.rename(columns={'Runtime':'Runtime_Min'},inplace=True) 
df['Released_Year'] = df['Released_Year'].astype(int)
df['Gross'] = df['Gross'].str.replace(r'[^0-9]','',regex=True)
df['Gross'] = df['Gross'].astype(float)

In [ ]:
y = base['IMDB_Rating']
X = base[['Released_Year','Runtime_Min','Meta_score','No_of_Votes','Gross']]
df_novo_filme = df[['Released_Year','Runtime_Min','Meta_score','No_of_Votes','Gross']]

In [ ]:
from sklearn.model_selection import train_test_split 
          
x_treino,x_teste,y_treino,y_teste = train_test_split(X,y)

In [ ]:
from sklearn import linear_model

regressao_linear = linear_model.LinearRegression()

regressao_linear.fit(x_treino,y_treino)
nota_regressao = regressao_linear.predict(x_teste)

In [ ]:
from sklearn.metrics import r2_score

metrica_r2_linear = r2_score(y_teste,nota_regressao)

Para essa previsão eu escolhi originalmente as colunas: Released_Year,Runtime_Min e Meta_score para que pudessem ser usadas no treinamento dos modelos as transformando em variaveis númericas, visto que os modelos geralmente trabalham com números. A coluna que originalmente se chamava Runtime que apresentava a duração do filme em formato de string foi convertida para um número inteiro e por questões de clareza decidi mudar o nome para Runtime_Min, indicando a duração em minutos.

O problema que estamos resolvendo é de REGRESSÃO, pois queremos prever um valor númerico, que é a nota IMDb de um filme.

Entre os modelos que testei (Huber Regressor, Árvore de Decisão e Regressão Linear) o que teve a melhor performance foi o modelo de Regressão Linear, 
entendendo mais de 40% das variações das notas, enquanto os outros não entendiam nem 20%. A Regressão Linear apresentou essa performance após a adição das colunas No_of_Votes e Gross. Como próximo passo, implementaria a coluna de gênero (que precisaria de um tratamento especial para que o modelo entedesse, como variaveis dummys, por exemplo) para que os resultados melhorassem ainda mais.

O modelo de Regressão Linear é um modelo mais simples de usar e muito claro, ajudando até mesmo a entender quais colunas podem ser mais importantes para começar uma análise usando o coef_. Seu treinamento é rápido por ser mais simples. 

Porém, justamente por ser um modelo mais simples, pode não performar bem com dados mais complexos bem como outliers, que são valores extremos(para baixo ou para cima).

O r2_score é mais completo na análise e o seu resultado é melhor e mais claro de se interpretar. 

## Exportando o Modelo

In [ ]:
CAMINHO_MODELO = '../models/'
NOME_ARQUIVO = 'modelo_regressao_final.pkl'
os.makedirs(CAMINHO_MODELO, exist_ok=True)

joblib.dump(regressao_linear, CAMINHO_MODELO + NOME_ARQUIVO)

## Usando o Modelo

In [ ]:
modelo_carregado = joblib.load('../models/modelo_regressao_final.pkl')
predicoes = modelo_carregado.predict(df_novo_filme)
print(f'A nota IMDb prevista para o filme "The Shawshank Redemption" é de: {predicoes[0]:.2f}')